In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.13.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


## Exercise 1

Compute a Gröbner basis $G$ of the ideal $I$ w.r.t. the lexicographic order.

In [331]:
R, (x,y,z) = polynomial_ring(QQ,["x","y","z"])

f1 = x^2 + y^2 + z^2 - 4
f2 = x^2 + 2*y^2 - 5
f3 = x*z - 1

I = ideal([f1,f2,f3])
G = groebner_basis(I, ordering=lex(R), complete_reduction=true)

Gröbner basis with elements
1 -> 2*z^4 - 3*z^2 + 1
2 -> y^2 - z^2 - 1
3 -> x + 2*z^3 - 3*z
with respect to the ordering
lex([x, y, z])

Note that the first polynomial only involves a single variable $z$. 

In [332]:
G[1]

2*z^4 - 3*z^2 + 1

Since this polynomial has only one variable, we can solve it in Maple, WolframAlpha or whatever other tool you're familiar with! We then attempt to solve for the other variables, using the other polynomials in the Gröbner basis. 

If we like, we can also try so solve it over the complex numbers, either exactly or numerically using these tools.

## Solving over $\mathbb{Q}$ in Oscar

The command for finding root of **univariate** polynomials in Oscar is `roots()`. 

In our case there is a subtlety though: Oscar doesn't see $2z^4 - 3z^2 + 1$ as a univariate polynomial! Instead, it sees it as a member of the multivariate polynomial ring $\mathbb{Q}[x,y,z]$. Our first step is therefore to convert it to a **univariate** polynomial, in some univariate polynomial ring $\mathbb{Q}[t]$, using the `to_univariate` function.

In [333]:
#Create a univariate polynomial ring with rational coefficients
UR, t = polynomial_ring(QQ, "t")

(Univariate polynomial ring in t over QQ, t)

In [334]:
g1_univariate = to_univariate(UR,G[1])

2*t^4 - 3*t^2 + 1

Note the difference in where Oscar thinks these polynomials live!

In [335]:
parent(G[1])

Multivariate polynomial ring in 3 variables x, y, z
  over rational field

In [336]:
parent(g1_univariate)

Univariate polynomial ring in t over QQ

Let's now find the rational solutions of $p$:

In [339]:
sol_z = roots(g1_univariate, QQ)

2-element Vector{QQFieldElem}:
 -1
 1

There are two such solutions, which are values for z. We now find the values of $x$ and $y$ going up. 

We use the second entry of the Gröbner basis, to find the value of $y$:

In [340]:
p1 = G[2](x,y,sol_z[1])

y^2 - 2

In [341]:
roots(to_univariate(UR,p1),QQ)

QQFieldElem[]

In [342]:
p2 = G[2](x,y,sol_z[2])

y^2 - 2

In [343]:
roots(to_univariate(UR,p2),QQ)

QQFieldElem[]

**Conclusion:** There are no rational solutions! 

## Bonus: Solutions over $\mathbb{C}$

If we want to solve over the complex numbers instead, we have to do a bit more work in Oscar.

First, we create a version of $\mathbb{C}$. This is a bit subtle! Oscar can't really deal with neither $\mathbb{R}$ nor $\mathbb{C}$ since equivalence classes of Cauchy sequences or Dedekind cuts (or whatever you favorite way of defining $\mathbb{R}$ is) are very hard to set up in a computer.

Instead, Oscar works with **intervals** of **floating point numbers** with a certain precison (e.g., 32 bits).

In [344]:
CC = AcbField(32) #Complex field with precision 32

Complex Field with 32 bits of precision and error bounds

Our first order of business is to reinterpret the polynomials in the Gröbner basis as polynomials over this version of $\mathbb{C}$, instead of $\mathbb{Q}$

In [350]:
g1 = change_base_ring(CC,G[1])
g2 = change_base_ring(CC,G[2])
g3 = change_base_ring(CC,G[3])

x + 2.000000000*z^3 - 3.000000000*z

These polynomials now live in $\mathbb{C}[x,y,z]$. To make sure that the variables $x,y,z$ are now understood as variables in this ring, we have to write the following:

In [354]:
(x,y,z) = gens(parent(g3))

3-element Vector{AbstractAlgebra.Generic.MPoly{acb}}:
 x
 y
 z

Let's now solve $g_1$ over this $\mathbb{C}$! Remember to first convert it to a univariate polynomial, this time over $\mathbb{C}$.

In [356]:
UR_CC, t = polynomial_ring(CC,"t")

(Univariate Polynomial Ring in t over Complex Field with 32 bits of precision and error bounds, t)

2.000000000*t^4 - 3.000000000*t^2 + 1

In [362]:
sol_z = roots(to_univariate(UR_CC,g1),CC)

4-element Vector{acb}:
 [-1.0000000 +/- 5.36e-9]
 [-0.70710678 +/- 4.10e-9]
 [0.70710678 +/- 4.34e-9]
 [1.00000000 +/- 3.73e-9]

Note that `0.707106781 +/- 2.74e-10` means that the number is just an approximation with possible error $ \pm 2.74\cdot 10^{-10}$

Next we want to plug in these solutions in the other polynomials we found in our Gröbner basis.

The first value of $z$ gives the following polynomials:

In [375]:
g21 = g2(x,y,sol_z[1])
println(g21)

g31 = g3(x,y,sol_z[1])
println(g31)

y^2 + [-2.0000000 +/- 1.10e-8]
x + [1.0000000 +/- 4.45e-8]


Let's solve them:

In [389]:
println("y-solutions: ",roots(to_univariate(UR_CC,g21),CC))
println("x-solutions: ",roots(to_univariate(UR_CC,g31),CC))

y-solutions: acb[[-1.41421356 +/- 9.00e-9], [1.4142136 +/- 4.67e-8]]
x-solutions: acb[[-1.0000000 +/- 4.45e-8]]


Now we do the same for the other $z$-solutions:

In [390]:
g22 = g2(x,y,sol_z[2])
println(g22)
g32 = g3(x,y,sol_z[2])
println(g32)

println("y-solutions: ",roots(to_univariate(UR_CC,g22),CC))
println("x-solutions: ",roots(to_univariate(UR_CC,g32),CC))

y^2 + [-1.50000000 +/- 4.35e-9]
x + [1.4142136 +/- 5.52e-8]
y-solutions: acb[[-1.22474487 +/- 5.57e-9], [1.22474487 +/- 5.57e-9]]
x-solutions: acb[[-1.4142136 +/- 5.52e-8]]


In [391]:
g23 = g2(x,y,sol_z[3])
println(g23)
g33 = g3(x,y,sol_z[3])
println(g33)

println("y-solutions: ",roots(to_univariate(UR_CC,g23),CC))
println("x-solutions: ",roots(to_univariate(UR_CC,g33),CC))

y^2 + [-1.50000000 +/- 4.81e-9]
x + [-1.4142136 +/- 5.69e-8]
y-solutions: acb[[-1.22474487 +/- 5.95e-9], [1.22474487 +/- 5.95e-9]]
x-solutions: acb[[1.4142136 +/- 5.69e-8]]


In [392]:
g24 = g2(x,y,sol_z[4])
println(g24)
g34 = g3(x,y,sol_z[4])
println(g33)

println("y-solutions: ",roots(to_univariate(UR_CC,g24),CC))
println("x-solutions: ",roots(to_univariate(UR_CC,g34),CC))

y^2 + [-2.00000000 +/- 7.46e-9]
x + [-1.4142136 +/- 5.69e-8]
y-solutions: acb[[-1.41421356 +/- 8.32e-9], [1.41421356 +/- 8.51e-9]]
x-solutions: acb[[1.0000000 +/- 3.36e-8]]


**Conclusion:** For each root of z, there are two solutions in y and one of x. In total there are **8 solutions**.

We could have automated this last step a bit:

In [397]:
for z_value in sol_z
        println("z-solution: ", z_value)
        g2_evaluated = g2(x,y,z_value)
        println(g2_evaluated)
        g3_evaluated = g3(x,y,z_value)
        println(g3_evaluated)

        println("y-solutions: ",roots(to_univariate(UR_CC,g2_evaluated),CC))
        println("x-solutions: ",roots(to_univariate(UR_CC,g3_evaluated),CC))
        println()
end

z-solution: [-1.0000000 +/- 5.36e-9]
y^2 + [-2.0000000 +/- 1.10e-8]
x + [1.0000000 +/- 4.45e-8]
y-solutions: acb[[-1.41421356 +/- 9.00e-9], [1.4142136 +/- 4.67e-8]]
x-solutions: acb[[-1.0000000 +/- 4.45e-8]]

z-solution: [-0.70710678 +/- 4.10e-9]
y^2 + [-1.50000000 +/- 4.35e-9]
x + [1.4142136 +/- 5.52e-8]
y-solutions: acb[[-1.22474487 +/- 5.57e-9], [1.22474487 +/- 5.57e-9]]
x-solutions: acb[[-1.4142136 +/- 5.52e-8]]

z-solution: [0.70710678 +/- 4.34e-9]
y^2 + [-1.50000000 +/- 4.81e-9]
x + [-1.4142136 +/- 5.69e-8]
y-solutions: acb[[-1.22474487 +/- 5.95e-9], [1.22474487 +/- 5.95e-9]]
x-solutions: acb[[1.4142136 +/- 5.69e-8]]

z-solution: [1.00000000 +/- 3.73e-9]
y^2 + [-2.00000000 +/- 7.46e-9]
x + [-1.0000000 +/- 3.36e-8]
y-solutions: acb[[-1.41421356 +/- 8.32e-9], [1.41421356 +/- 8.51e-9]]
x-solutions: acb[[1.0000000 +/- 3.36e-8]]



## Exercise 2

### Part (a)

In [398]:
R, (x, y,z) = polynomial_ring(QQ, ["x", "y","z"])

(Multivariate polynomial ring in 3 variables over QQ, QQMPolyRingElem[x, y, z])

In [400]:
I = ideal([x^2+y^2+z^2-1, x^2+y^2+z^2-2*x, 2*x-3*y-z])
G = groebner_basis(I,ordering=lex(gens(R)))

Gröbner basis with elements
1 -> 40*z^2 - 8*z - 23
2 -> 3*y + z - 1
3 -> 2*x - 3*y - z
with respect to the ordering
lex([x, y, z])

At this point, we can switch to solving the system in Maple!

But we can also try to do it in Oscar, using the commands we learned above in Problem 1.

In [402]:
CC = AcbField(32) #Complex field with precision 32
g1 = change_base_ring(CC,G[1])
g2 = change_base_ring(CC,G[2])
g3 = change_base_ring(CC,G[3])

2.000000000*x - 3.000000000*y - 1.000000000*z

In [403]:
#Create a univariate polynomial ring with CC coefficients
UR, t = polynomial_ring(CC, "t")
sol_z = roots( to_univariate(UR, change_base_ring(CC,G[1])), CC)

2-element Vector{acb}:
 [-0.664852927 +/- 6.76e-10]
 [0.86485293 +/- 3.75e-9]

There are two solutions in z. For each solution, we need to find the x and y solutions. 

In [404]:
(x,y,z) = gens(parent(g2))
g21 = g2(x,y,sol_z[1])
g22 = g2(x,y,sol_z[2])

3.000000000*y + [-0.13514707 +/- 3.75e-9]

In [405]:
y1=roots(to_univariate(UR,g21),CC)

1-element Vector{acb}:
 [0.55495098 +/- 5.30e-9]

In [406]:
y2=roots(to_univariate(UR,g22),CC)

1-element Vector{acb}:
 [0.045049024 +/- 5.99e-10]

These are the two solutions in y. We now compute x:

In [407]:
(x,y,z) = gens(parent(g3))
g31 = g3(x,y1[1],sol_z[1])
g32 = g3(x,y2[1],sol_z[2])

2.000000000*x + [-1.0000000 +/- 2.44e-9]

In [408]:
x1 = roots(to_univariate(UR,g31))
println(x1)

x2 = roots(to_univariate(UR,g32))
println(x2)

acb[[0.50000000 +/- 1.76e-9]]
acb[[0.50000000 +/- 1.22e-9]]


List the points on the variety

In [409]:
println([sol_z[1],y1[1],x1[1]])
println([sol_z[2],y2[1],x2[1]])

acb[[-0.664852927 +/- 6.76e-10], [0.55495098 +/- 5.30e-9], [0.50000000 +/- 1.76e-9]]
acb[[0.86485293 +/- 3.75e-9], [0.045049024 +/- 5.99e-10], [0.50000000 +/- 1.22e-9]]


### Part (b)

In [411]:
R, (x, y,z) = polynomial_ring(QQ, ["x", "y","z"])

I = ideal([x^2*y-z^3, 2*x*y-4*z-1,z-y^2,x^3-4*z*y])
G = groebner_basis(I, ordering=lex(R), complete_reduction=true)

Gröbner basis with elements
1 -> 1
with respect to the ordering
lex([x, y, z])

The ideal contains the constant polynomial $1$, so the variety is empty!